# Part 01: Import and export the street data
michael babb  
2025 02 18

In [1]:
# standard
import os

In [2]:
# external
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd

In [3]:
# custom
from geo_data_io.fc_df_spatial import write_gdf
import run_constants as rc
from utils import create_city_sector, create_name


# load the street network shapefile as a gpd

In [4]:
# setup the output paths
if not os.path.exists(rc.OUTPUT_FILE_PATH):
    os.makedirs(rc.OUTPUT_FILE_PATH)

In [5]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S01_SND_IN_FILE_NAME)

In [6]:
gdf = gpd.read_file(filename = fpn)

c:\Users\babbm\anaconda3\envs\analysis\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column SNDSEG_UPDATE, 2007-04-22T00:00:00.0Z, successfully parsed
  return ogr_read(


In [7]:
gdf.shape

(34378, 37)

# lower case and rename columns

In [8]:
gdf.columns = [cn.lower() for cn in gdf.columns]

In [9]:
gdf.columns

Index(['f_intr_id', 't_intr_id', 'snd_id', 'snd_feacode', 'citycode',
       'stname_id', 'st_code', 'arterial_code', 'segment_type', 'agency_code',
       'access_code', 'divided_code', 'structure_type', 'legalloc_code',
       'vehicle_use_code', 'gis_seg_length', 'l_adrs_from', 'l_adrs_to',
       'r_adrs_from', 'r_adrs_to', 'ord_pre_dir', 'ord_street_name',
       'ord_street_type', 'ord_suf_dir', 'ord_stname_concat', 'l_city',
       'l_state', 'l_zip', 'r_city', 'r_state', 'r_zip', 'sndseg_update',
       'compkey', 'comptype', 'unitid', 'unitid2', 'geometry'],
      dtype='object')

# change datatypes

In [10]:
for cn in gdf.columns:
    if gdf[cn].dtype == np.int32:
        gdf[cn] = gdf[cn].astype(np.int64)
    

# indicate the sector of the city the street is in

In [11]:
# combine the ord_pre_dir and the ord_suf_dir to get the sector of the city.

In [12]:
col_names = ['ord_pre_dir', 'ord_suf_dir']

In [13]:
for cn in col_names:
    gdf[cn] = gdf[cn].fillna('')

In [14]:
gdf['city_sector'] = gdf.apply(func = create_city_sector, axis = 1)

In [15]:
gdf['city_sector'].unique()

array(['S', 'E', 'NE', 'NW', 'N', 'CNTR', 'SW', 'W'], dtype=object)

## note: I used qGIS to manually recode the following streets:
MAGNOLIA BR WB: W  
WEST SEATTLE BR WB: mix of SW and S  
WEST SEATTLE BR EB: mix of SW and S  
1ST AV S BR SB: mix of SW and S  
1ST AV S BR NB: mix of SW and S  
(it's a lot easier to do this in qGIS)

In [16]:
check = gdf.loc[gdf['city_sector'].str[-1] == 'B', 'ord_stname_concat'].unique().tolist()

In [17]:
check

[]

In [18]:
# let's create a unique name field

In [19]:
gdf['ord_stname_unique'] = gdf.apply(func = create_name, axis = 1)

In [20]:
gdf['ord_stname_concat'].unique().shape

(3692,)

In [21]:
gdf['ord_stname_unique'].unique().shape

(3696,)

In [22]:
# let's check what type of geometry we are working with
gdf['geometry'].geom_type.value_counts()

MultiLineString    34378
Name: count, dtype: int64

In [23]:
# everything is a MultLineString??? is that necessary?
# we can call the gdf.explode() function to convert MultiLineStrings to LineStrings
# if the record counts are the same, we can convert everything to LineStrings 
test_gdf = gdf.explode()

In [24]:
test_gdf.shape

(34378, 39)

In [25]:
# nope! The number of records is the same, indicating that MultiLineStrings are not necessary
gdf = test_gdf.copy()

In [26]:
gdf['geometry'].geom_type.value_counts()

LineString    34378
Name: count, dtype: int64

# write the geodataframe to disk as a gpkg

In [27]:
# reproject to wgs 84
gdf = gdf.to_crs(crs = 'epsg:4326')

In [28]:
write_gdf(gdf = gdf, output_file_path=rc.OUTPUT_FILE_PATH,
           output_file_name=rc.S01_SND_OUT_FILE_NAME)

# CREATE A GRAPH

In [30]:
gdf.head()

,f_intr_id,t_intr_id,snd_id,snd_feacode,citycode,stname_id,st_code,arterial_code,segment_type,agency_code,...,r_state,r_zip,sndseg_update,compkey,comptype,unitid,unitid2,city_sector,ord_stname_unique,geometry
0,17740,17791,33478,5,1,566,0,1,1,1,...,WA,98108,2007-04-22 00:00:00+00:00,8460,68,04805,0820,S,8TH AVE S,"LINESTRING (-122.32287 47.52982, -122.32289 47..."
1,11391,11295,20097,1,1,659,0,0,1,1,...,WA,98102,2007-04-22 00:00:00+00:00,9236,68,05465,0010,E,BELMONT AVE E,"LINESTRING (-122.32402 47.61849, -122.32403 47..."
2,982,979,42208,1,0,1797,40,0,1,3,...,WA,98155,2004-05-19 00:00:00+00:00,0,0,0,0,NE,NE 187TH CT,"LINESTRING (-122.29193 47.7649, -122.29127 47...."
3,11695,11696,20460,77,1,1444,20,0,6,1,...,WA,98122,2004-05-19 00:00:00+00:00,0,0,0,0,E,E PIKE ST,"LINESTRING (-122.30782 47.61411, -122.30749 47..."
4,6257,6252,9237,1,1,2086,0,0,1,1,...,WA,98107,2012-10-12 00:00:00+00:00,18575,68,14630,0200,NW,NW 63RD ST,"LINESTRING (-122.38214 47.67456, -122.38476 47..."


In [31]:
sg = nx.from_pandas_edgelist(df = gdf,
source = 'f_intr_id', target = 't_intr_id', edge_attr = True)

In [32]:
sg.has_edge(u = 7710, v = 7711)

True

# export the blank street types to an excel file for manual inspection

In [ ]:
gdf['ord_street_type'].unique()

In [ ]:
blank_street_type_df = gdf.loc[gdf['ord_street_type'].isna(), 'ord_stname_concat'].drop_duplicates().to_frame()

In [ ]:
blank_street_type_df.shape

In [ ]:
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S01_BST_OUT_FILE_NAME)

In [ ]:
blank_street_type_df.to_excel(excel_writer=ofpn, index = False)